In [ ]:
import pandas as pd
import numpy as np
import time
import torchvision
from torchvision import models
import torch.nn as nn
from tqdm import tqdm_notebook as tqdm
from sklearn.model_selection import train_test_split, StratifiedKFold
from PIL import Image, ImageFile
from torch.utils.data import Dataset
import torch
import torch.optim as optim
from torchvision import transforms
from torch.optim import lr_scheduler
import os
import random
import cv2
import copy
import matplotlib.pyplot as plt
%matplotlib inline
from imgaug import augmenters as iaa
from skimage import io, transform
from os.path import isfile
import sys
import os

package_path = '../input/efficientnet/efficientnet-pytorch/EfficientNet-PyTorch/'
sys.path.append(package_path)
from efficientnet_pytorch import EfficientNet

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

In [ ]:
def imshow(inp, title=None):
    """Imshow for Tensor."""
    plt.figure(figsize=(12,12))
    inp = inp.numpy().transpose((1, 2, 0))
    #undo normalising
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp,0,1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated
    
def crop_image_from_gray(img,tol=7):
    if img.ndim ==2:
        mask = img>tol
        return img[np.ix_(mask.any(1),mask.any(0))]
    elif img.ndim==3:
        gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        mask = gray_img>tol
        
        check_shape = img[:,:,0][np.ix_(mask.any(1),mask.any(0))].shape[0]
        if (check_shape == 0): # image is too dark so that we crop out everything,
            return img # return original image
        else:
            img1=img[:,:,0][np.ix_(mask.any(1),mask.any(0))]
            img2=img[:,:,1][np.ix_(mask.any(1),mask.any(0))]
            img3=img[:,:,2][np.ix_(mask.any(1),mask.any(0))]
            img = np.stack([img1,img2,img3],axis=-1)
        return img

def gamma(image,g=1.00):
    invGamma = 1.0 / g
    table = np.array([((i / 255.0) ** invGamma) * 255
        for i in np.arange(0, 256)]).astype("uint8")
    return cv2.LUT(image, table)


class MyEyeDataset(Dataset):

    def __init__(self, dataframe, training=True, transform=None, sigmax=10):
        self.df = dataframe
        self.transform = transform
        self.training = training
        self.sigmax = sigmax
    
    def __len__(self):
        return len(self.df)
   
    def __getitem__(self, idx):
        if self.training:
            img_name = os.path.join(input_dir, 'train_images', self.df.loc[idx, 'id_code'] + '.png')
        else:    
            img_name = os.path.join(input_dir, 'test_images', self.df.loc[idx, 'id_code'] + '.png')
            
        image = cv2.imread(img_name)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = crop_image_from_gray(image)
                
        image = transforms.ToPILImage()(image)
        
        if self.transform:
            image = self.transform(image)
        label = torch.tensor(self.df.loc[idx, 'diagnosis'])
        
        return image, label

In [ ]:
model_name = 'efficientnet-b4'
#model_weight = 'drdrdr/31_b3_new_data.bin'
IMG_SIZE    = 256
TTA         = 1
n_folds = 5
model_folder = 'model45'
fold_model_name = '45_b4_new_data_fold'
BS = 4
device = torch.device("cuda:0")
input_dir = '../input/aptos2019-blindness-detection'
sample = pd.read_csv('../input/aptos2019-blindness-detection/sample_submission.csv')
num_classes = 1


In [ ]:

test_tsfm = transforms.Compose([
                           transforms.Resize((IMG_SIZE,IMG_SIZE), interpolation=2),
#                            transforms.RandomHorizontalFlip(),
#                            transforms.RandomVerticalFlip(),
#                            transforms.RandomRotation(360),
#                            transforms.ColorJitter(contrast=(1,2)),
                           transforms.ToTensor(),
                           transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
                                              ])

test_set       = MyEyeDataset(sample, transform = test_tsfm, training=False, sigmax=25)
test_loader   = torch.utils.data.DataLoader(test_set, batch_size=BS, shuffle=False, num_workers=4)


In [ ]:
# example_loader = torch.utils.data.DataLoader(test_set, batch_size=4, shuffle=True, num_workers=4)

# class_names = [0,1,2,3,4]
# # Get a batch of training data
# inputs, classes = next(iter(example_loader))

# # Make a grid from batch
# out = torchvision.utils.make_grid(inputs)

# imshow(out, title=[class_names[x] for x in classes])

In [ ]:
sample = pd.read_csv('../input/aptos2019-blindness-detection/sample_submission.csv')
test_pred = np.zeros((len(sample), 1))

for i in range(n_folds):
    fold_model = EfficientNet.from_name(model_name)
    in_features = fold_model._fc.in_features
    fold_model._fc = nn.Linear(in_features, num_classes)
    path = f'../input/{model_folder}/{fold_model_name}_{i+1}.bin'
    fold_model.load_state_dict(torch.load(path))
    fold_model.cuda()

    for param in fold_model.parameters():
        param.requires_grad = False
        
    fold_model.eval()
        

    with torch.no_grad():
        for i, data in tqdm(enumerate(test_loader)):
            images, _ = data
            images = images.cuda()
            pred = fold_model(images)
            test_pred[i * BS:(i + 1) * BS] += pred.detach().cpu().squeeze().numpy().reshape(-1, 1)

       
output = test_pred/n_folds

In [ ]:
output

In [ ]:
coef = [0.52544054, 1.54382343, 2.46199097, 3.30273553]
#coef = [0.51114902, 1.4533247 , 2.59416676, 3.46693649]
#coef = [0.52311745, 1.43932228, 2.60883522, 3.44006271]

for i, pred in enumerate(output):
    if pred < coef[0]:
        output[i] = 0
    elif pred >= coef[0] and pred < coef[1]:
        output[i] = 1
    elif pred >= coef[1] and pred < coef[2]:
        output[i] = 2
    elif pred >= coef[2] and pred < coef[3]:
        output[i] = 3
    else:
        output[i] = 4


sample = pd.read_csv("../input/aptos2019-blindness-detection/sample_submission.csv")
sample.diagnosis = output.astype(int)
sample.to_csv("submission.csv", index=False)

In [ ]:
sample.head(20)


In [ ]:
sample.groupby('diagnosis')['id_code'].count()/sample.shape[0]
